In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = response = ollama_via_openai.chat.completions.create(
    model="llama3.2",
    messages=[{"role": "user", "content": message}]
)
print(response.choices[0].message.content)

It's great to meet you too! I'm thrilled to be your conversational AI assistant. Welcome to our virtual chat! I'll do my best to help answer any questions, share interesting facts, or simply chat with you about a topic that interests you.

To get us started, how would you like our conversation to unfold? Would you like me to suggest some topics or games, or is there something on your mind that you'd love to talk about?


In [5]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [7]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [8]:
website_url = Website("https://wolf.org/wolf-info/basic-wolf-info/biology-and-behavior/")
print(website_url.title)
print(website_url.text)

Wolf Biology and Behavior | International Wolf Center
Wolf Info
Learn About Wolves
Wolf FAQ’s
Types of Wolves
Biology and Behavior
Wolves and Humans
In-Depth Resources
Fact or Scat Trivia
Wolves of the World
United States
Canada
Europe
Asia
Africa
Magazine
Magazine Archives
Wolf Ethogram
Facts vs Fiction
Just For Kids
Vocabulary
Fun Wolf Facts
Wolf Families
Wild Kids Articles
The Coloring Den
WolfQuest
Our Wolves
Meet Our Wolves
Exhibit Pack Log
Retired Pack Log
Wolf Cams
Wolf Pup Cam
Videos
Visit
Accessibility
Plan Your Visit
Group Visits
Programs
Adventure Programs
School Programs
Webinars
WolfLink Virtual Learning for adults & groups
Traveling Exhibit
Dr. L. David Mech Fellowship
2026 International Wolf Symposium
About
Mission & Vision
Diversity, Equity and Inclusion
Land Acknowledgment
Subscribe
History
Board of Directors
Employment
Internships
Contact Us
Support
Donate
Become a Member
Join the Legacy Pack
Other Ways to Support
Wolf Guardian Sponsorship
Wolf Care Auction
Volunteer


In [9]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [10]:
messages_for(website_url)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Wolf Biology and Behavior | International Wolf Center\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nWolf Info\nLearn About Wolves\nWolf FAQ’s\nTypes of Wolves\nBiology and Behavior\nWolves and Humans\nIn-Depth Resources\nFact or Scat Trivia\nWolves of the World\nUnited States\nCanada\nEurope\nAsia\nAfrica\nMagazine\nMagazine Archives\nWolf Ethogram\nFacts vs Fiction\nJust For Kids\nVocabulary\nFun Wolf Facts\nWolf Families\nWild Kids Articles\nThe Coloring Den\nWolfQuest\nOur Wolves\nMeet Our Wolves\nExhibit Pack Log\nRetired Pack Log\nWolf Cams\nWolf Pup Cam\nVideos\nVisit\nAccessibility\nPlan Your Vi

In [11]:
import ollama

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
    model="llama3.2",
    messages=messages_for(website)
)
    return response.choices[0].message.content

In [12]:
summarize("https://wolf.org/wolf-info/basic-wolf-info/biology-and-behavior/")

'**Wolf Biology and Behavior**\n==========================\n\nThe International Wolf Center provides comprehensive information on wolf biology and behavior, covering various aspects of their ecology, social structure, habitat requirements, and human-wolf interactions.\n\n### Overview\n\nThe website features articles, guides, and resources for learning about wolves, including their communication patterns, reproduction, pup development, adult survival, and dispersal habits. It also provides information on hunting and feeding behaviors, prey- predator dynamics, population cycles, and potential changes in wolf populations.\n\n### Notable Topics\n\n* **Migration and Dispersal**: Young wolves typically disperse at 1-2 years old to find a mate, territory, and start a new pack.\n* **Hunting and Feeding Behavior**: Wolves primarily feed on prey animals larger than themselves, using a "feast or famine" lifestyle with periods of little food followed by gorging on multiple prey items when hunting 

In [13]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [14]:
display_summary("https://wolf.org/wolf-info/basic-wolf-info/biology-and-behavior/")

**Summary of Wolf Biology and Behavior**
=====================================

The International Wolf Center website provides in-depth information on wolf biology and behavior. Here's a summary of the key topics:

### Biology

* Wolves can survive on 2.5 pounds of food per day, but require about five to seven pounds per day to reproduce successfully.
* They are known to live up to 13 years in the wild and 16 years in captivity.
* Gray wolves inhabit an area much wider than their prey species, with different populations preying on different animals.

### Behavior

* Dispersal is the primary way wolves colonize new areas and maintain genetic diversity. Wolves typically disperse when 1-2 years old as they reach sexual maturity.
* Young wolves often leave home at the age of two or three to find a mate and form a new pack.
* Wolf habitats can occur anywhere with sufficient large ungulates, such as deer, moose, elk, caribou, bison, and musk ox.

### Habitats and Prey

* The number of individuals per pack varies widely from 4-8 in the western Great Lakes area to up to 30 or more in parts of Canada and Alaska.
* Pack sizes are typically between 25-100 miles in the western Great Lakes area.
* Wolf kill rates vary with winter severity, but can keep prey populations at low levels for extended periods.

### Impacts on Prey

* Wolf predation has played a crucial role in shaping ungulate herds, making them healthier and more resilient to disease and genetic disorders.
* However, wolf-human conflicts can occur if human-caused mortality is high (greater than 28-50% of the fall wolf population).

**News and Announcements**
---------------------------

The website features various news headlines, including:

* "Birth of 4 Mexican Wolf Pups Brings Hope for the Endangered Canines" (June 27, 2025)
* "Montana Releases Final Wolf Management Plan"
* "Sweden’s Wolves in Hunters’ Sights as Government Eases Protection"

These announcements highlight the ongoing efforts to conserve and manage wolf populations around the world.